In [ ]:
from IPython.display import clear_output
import SALib
from SALib.sample import saltelli
from wolf_sheep.model import WolfSheep
from wolf_sheep.agents import Wolf, Sheep
from mesa.batchrunner import BatchRunner
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from model import *
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pprint import pprint

%matplotlib inline

In [37]:
problem = {
    'num_vars': 8,
    'names': ['international_trade', 'max_eff', 'eutax','neighbor_influence','tax_influence',
              'member_trade_multiplier','randomness','eu_strategy'],
    'bounds': [[0, 1], [0.01, 0.1], [5, 30], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]
}

#international_trade, max_eff, eutax, neighbor_influence,
#tax_influence, member_trade_multiplier, randomness, eu_strategy):
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 2
max_steps = 1
distinct_samples = 1

'''
"other_count":"other_count",
            "average_cooperativeness":"average_cooperativeness",
            "other_wealth":"other_wealth",
            "total_wealth":"total_wealth",
            "member_wealth":"member_wealth",
            "other_eff":"other_eff",
            "total_eff":"total_eff",
            "member_eff":"member_eff"
'''
# Set the outputs
model_reporters = {"Average cooperativeness": lambda m: m.datacollector.get_model_vars_dataframe()["average_cooperativeness"].values[1:],
                   "EU count": lambda m: m.datacollector.get_model_vars_dataframe()["member_count"].values[1:],
                   "Non-EU wealth": lambda m: m.datacollector.get_model_vars_dataframe()["other_wealth"].values[1:],
                   "Total wealth": lambda m: m.datacollector.get_model_vars_dataframe()["total_wealth"].values[1:],
                   "EU wealth": lambda m: m.datacollector.get_model_vars_dataframe()["member_wealth"].values[1:],
                   "Non-EU efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["other_eff"].values[1:],
                   "Total efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["total_eff"].values[1:],
                   "EU-efficiency": lambda m: m.datacollector.get_model_vars_dataframe()["member_eff"].values[1:],
                  }

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)

# READ NOTE BELOW CODE
batch = BatchRunner(RegionModel, 
                    max_steps=max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['international_trade', 'max_eff', 'eutax','neighbor_influence','tax_influence',
                                         'member_trade_multiplier','randomness','eu_strategy'])
                                    #'average_cooperativeness', 'member_count', 'other_wealth', 'total_wealth' , 'member_wealth', 
                                    #    'other_eff','total_eff' ,'member_eff' ])
data['Run'], data['average_cooperativeness'], data['member_count'], data['other_wealth'], data['total_wealth'], data['member_wealth'] , data['other_eff'], data['total_eff'], data['member_eff'] = None, None, None, None, None, None,None, None, None
#data['Run'], data['average_cooperativeness'],  data['member_count'], data['other_wealth'], data['total_wealth'], data['member_wealth'], data['other_eff'] , data['total_eff'], data['member_eff'] = None, None, None, None, None, None,None, None, None

In [38]:
# Change this for extra strategies
for i, param_val in enumerate(param_values):
    param_values[i, 7] =  0 if param_values[i, 7] < 0.5 else 1
    param_values[i, 0] =  0 if param_values[i, 0] < 0.5 else 1
for i in range(replicates):
    for j, vals in enumerate(param_values):
        vals = list(vals)
        vals[2] = int(vals[2])
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val

        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count # Don't know what causes this, but iteration number is not correctly filled
        data.iloc[count, 0:8] = vals
        data.iloc[count, 8:17] = iteration_data
        count += 1
        
        clear_output()
        
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')

100.00% done


In [39]:
data

,international_trade,max_eff,eutax,neighbor_influence,tax_influence,member_trade_multiplier,randomness,eu_strategy,Run,average_cooperativeness,member_count,other_wealth,total_wealth,member_wealth,other_eff,total_eff,member_eff
0,0.0,0.018701,17,0.676758,0.280273,0.907227,0.045898,1.0,0,[-0.0401682041142225],[152],[1.0105644452321842],[0.009359768393809415],[0.009414796647335924],[1.0106761803758846],[0.009388658226910832],[1.0106231061826267]
1,1.0,0.018701,17,0.676758,0.280273,0.907227,0.045898,1.0,1,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
2,0.0,0.016240,17,0.676758,0.280273,0.907227,0.045898,1.0,2,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
3,0.0,0.018701,7,0.676758,0.280273,0.907227,0.045898,1.0,3,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
4,0.0,0.018701,17,0.254883,0.280273,0.907227,0.045898,1.0,4,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
5,0.0,0.018701,17,0.676758,0.161133,0.907227,0.045898,1.0,5,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
6,0.0,0.018701,17,0.676758,0.280273,0.383789,0.045898,1.0,6,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
7,0.0,0.018701,17,0.676758,0.280273,0.907227,0.143555,1.0,7,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
8,0.0,0.018701,17,0.676758,0.280273,0.907227,0.045898,0.0,8,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
9,0.0,0.016240,7,0.254883,0.161133,0.383789,0.143555,0.0,9,[-0.02639794329835531],[152],[1.0101278410075052],[0.008899755629848299],[0.009632094978440501],[1.0109241550935426],[0.009284233787859206],[1.0105459059026747]
